In [1]:
print('hello')

hello


In [1]:
import numpy as np

# Define the input values (homework and exam scores)
# Define the input values (homework and exam scores)
homework_score = 50
exam_score = 50
# Define the fuzzy sets for homework and exam scores
homework_low = [0, 30, 60]
homework_medium = [30, 70, 100]
homework_high = [70, 100, 100]

exam_low = [0, 30, 60]
exam_medium = [30, 70, 100]
exam_high = [70, 100, 100]
# Define the input membership function

def fuzzy_membership(value, low, medium, high):
 return [
     max(0, min((value - low) / (medium - low), (high - value) / (high - medium))),
     max(0, min((value - low) / ((high - low) / 2), 2 - (value - low) / ((high - low) / 2))),
     max(0, min(1, (value - medium) / (high - medium)))
 ]
    
# Calculate the membership values for the input values
homework_membership = fuzzy_membership(homework_score, *homework_low)
exam_membership = fuzzy_membership(exam_score, *exam_low)
# Define the fuzzy rules and their corresponding linear functions
rules = [
 (homework_membership[0], exam_membership[0], lambda x, y: 0.5 * x + 0.5 * y),
 (homework_membership[0], exam_membership[1], lambda x, y: 0.5 * x + 0.5 * y),
 (homework_membership[0], exam_membership[2], lambda x, y: 0.5 * x + 0.5 * y),
 (homework_membership[1], exam_membership[0], lambda x, y: 0.5 * x + 0.5 * y),
 (homework_membership[1], exam_membership[1], lambda x, y: 0.5 * x + 0.5 * y),
 (homework_membership[1], exam_membership[2], lambda x, y: 0.5 * x + 0.5 * y),
 (homework_membership[2], exam_membership[0], lambda x, y: 0.5 * x + 0.5 * y),
 (homework_membership[2], exam_membership[1], lambda x, y: 0.5 * x + 0.5 * y),
 (homework_membership[2], exam_membership[2], lambda x, y: 0.5 * x + 0.5 * y),
]

# Calculate the weighted average of the linear functions
numerator = sum(weight * rule(homework_score, exam_score) for weight, _, rule in rules)
denominator = sum(weight for weight, _, _ in rules)

# Calculate the final result (Sugeno model)
if denominator != 0:
    result = numerator / denominator
else:
    result = 0 # Handle the case where the denominator is 0
if result <= 40:
    output_grade = "Low"
elif result <= 70:
    output_grade = "Medium"
else:
    output_grade = "High"
# Output the recommended result (in this case, a grade)
print("Recommended Grade:", output_grade)

Recommended Grade: Medium


In [6]:
import numpy as np

# Define the input values
biaya_produksi_score = 6000000
permintaan_score = 320
stok_score = 60
jumlah_produksi_score = 0

# Modified fuzzy sets with normalized ranges
biaya_produksi_minimum = [0, 450000, 900000]
biaya_produksi_maksimum = [450000, 910000, 16600000]
permintaan_minimum = [0, 25, 50]
permintaan_maksimum = [25, 60, 485]
stok_minimum = [0, 2, 5]
stok_maksimum = [2, 6, 230]
jumlah_produksi_minimum = [0, 40, 80]
jumlah_produksi_maksimum = [40, 90, 472]

# Normalize function to keep values in appropriate ranges
def normalize_value(value, min_val, max_val, target_min, target_max):
    return ((value - min_val) / (max_val - min_val)) * (target_max - target_min) + target_min

# Define the input membership function
def fuzzy_membership(value, low, medium, high):
    return [
        max(0, min((value - low) / (medium - low), (high - value) / (high - medium))),
        max(0, min((value - medium) / (high - medium), (high - value) / (high - medium))),
        max(0, min(1, (value - medium) / (high - medium)))
    ]

# Calculate base production need
base_production_need = max(0, min(472, permintaan_score - stok_score))

# Normalize input values for calculation
normalized_biaya = normalize_value(biaya_produksi_score, 0, 16600000, 0, 100)
normalized_permintaan = normalize_value(permintaan_score, 0, 485, 0, 100)
normalized_stok = normalize_value(stok_score, 0, 230, 0, 100)

# Calculate membership values for each input
biaya_membership = fuzzy_membership(biaya_produksi_score, *biaya_produksi_minimum)
permintaan_membership = fuzzy_membership(permintaan_score, *permintaan_minimum)
stok_membership = fuzzy_membership(stok_score, *stok_minimum)

# Modified rules with adjusted multipliers to ensure output in range 260-320
rules = [
    (biaya_membership[0], permintaan_membership[0], stok_membership[0], 
     lambda x, y, z: min(472, base_production_need * 1.05)),  # 5% buffer for low cost
    
    (biaya_membership[0], permintaan_membership[1], stok_membership[0], 
     lambda x, y, z: min(472, base_production_need * 1.08)),  # 8% buffer for low cost, medium demand
    
    (biaya_membership[1], permintaan_membership[0], stok_membership[1], 
     lambda x, y, z: min(472, base_production_need * 1.02)),  # Small buffer for medium cost
    
    (biaya_membership[1], permintaan_membership[1], stok_membership[1], 
     lambda x, y, z: min(472, base_production_need * 1.10)),  # 10% buffer for balanced conditions
    
    (biaya_membership[1], permintaan_membership[2], stok_membership[1], 
     lambda x, y, z: min(472, base_production_need * 1.15)),  # Higher buffer for high demand
    
    (biaya_membership[2], permintaan_membership[1], stok_membership[2], 
     lambda x, y, z: min(472, base_production_need * 1.03)),  # Small buffer due to high cost
    
    (biaya_membership[2], permintaan_membership[2], stok_membership[2], 
     lambda x, y, z: min(472, base_production_need * 1.07))   # Moderate buffer balancing high cost/demand
]

# Calculate the weighted average of the linear functions
numerator = sum(weight * rule(biaya_produksi_score, permintaan_score, stok_score) 
                for weight, _, _, rule in rules)
denominator = sum(weight for weight, _, _, _ in rules)

# Calculate the final result (Sugeno model)
if denominator != 0:
    jumlah_produksi_score = min(472, numerator / denominator)
else:
    jumlah_produksi_score = 0

# Determine production level category
if jumlah_produksi_score <= jumlah_produksi_minimum[1]:
    output_level = "Low Production"
elif jumlah_produksi_score <= jumlah_produksi_maksimum[1]:
    output_level = "Medium Production"
else:
    output_level = "High Production"

# Output the recommended production level with integer display
print(f"Base production need (Demand - Stock): {int(base_production_need)}")
print(f"Recommended Production Score: {int(jumlah_produksi_score)}")
print(f"Production Level Category: {output_level}")

Base production need (Demand - Stock): 260
Recommended Production Score: 273
Production Level Category: High Production
